In [1]:
import pandas as pd
import urllib
from urllib import request
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import roc_auc_score
from sklearn import preprocessing
from sklearn.preprocessing import label_binarize
import numpy as np
import sklearn 
from random import seed
from random import randrange
from math import sqrt
import matplotlib.pyplot as plt
from sklearn import datasets, linear_model, metrics
from sklearn.metrics import mean_squared_error
from sklearn.cross_validation import train_test_split 
from sklearn.metrics import r2_score
from sklearn import datasets, linear_model, metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from subprocess import check_output
from datetime import time
from tsfresh import extract_features
from tsfresh.utilities.dataframe_functions import make_forecasting_frame
from sklearn.ensemble import AdaBoostRegressor
from tsfresh.utilities.dataframe_functions import impute
from tsfresh.feature_extraction import MinimalFCParameters
#selecting relavant features
from tsfresh import select_features
from tsfresh.utilities.dataframe_functions import impute

C:\Users\chethan\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
url = "https://raw.githubusercontent.com/LuisM78/Appliances-energy-prediction-data/master/energydata_complete.csv"
df = pd.read_csv(url)

In [3]:
df['date'] = pd.to_datetime(df['date'])
df['month'] , df['time'] , df['hour'] ,df['day'] , df['day_of_week'] =  df['date'].dt.month , df['date'].dt.time , df['date'].dt.hour , df['date'].dt.day , df['date'].dt.weekday_name
df['weekStatus'] = df['date'].dt.dayofweek
df['weekendstatus'] = np.where(df['weekStatus'] < 5, 0, 1)

#To Calculate NSM
df['datenew'] = df['date'].dt.date
datenum=df['datenew'].unique()
lennsm = len(datenum)+1
#print("len",lennsm)
data=[]
for j in range (1,lennsm):
    if (j==1):
        for i in range (0,42):
            if (i==0):
                a = 61200
                data.append(a)
            elif (i > 0 ):
                a = a + 600
                data.append(a)
    elif (j > 1 ):
        for i in range (0,144):
            if (i==0):
                a = 0
                data.append(a)
            elif (i > 0 ):
                a = a + 600
                data.append(a)
df1 = pd.DataFrame({'data_converted' : data})
dfc1 = df1[:len(df)]
df['NSM'] = dfc1
###################
d = df.date[0:len(df.date)]
data_final = []
for i in range (len(d)):
    if(i==0):
        a= 1
        data_final.append(a)
    elif(i>0):
        a=a+1
        data_final.append(a)
####################################
df["id"] = pd.DataFrame({'id':data_final})

df_day = pd.get_dummies(df['day_of_week'])
df = df.join(df_day)
del df['Monday']
del df['weekStatus']
del df['day_of_week']
del df['datenew']
del df['time']

In [4]:
df.tail(10)

,date,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,...,day,weekendstatus,NSM,id,Friday,Saturday,Sunday,Thursday,Tuesday,Wednesday
19725,2016-05-27 16:30:00,220,0,25.426667,46.060000,26.000000,41.700000,28.000000,40.760000,24.7,...,27,0,59400,19726,1,0,0,0,0,0
19726,2016-05-27 16:40:00,180,0,25.500000,46.530000,26.000000,41.725714,27.856667,40.500000,24.7,...,27,0,60000,19727,1,0,0,0,0,0
19727,2016-05-27 16:50:00,120,0,25.500000,47.456667,26.000000,42.320000,27.663333,40.693333,24.7,...,27,0,60600,19728,1,0,0,0,0,0
19728,2016-05-27 17:00:00,110,0,25.600000,47.193333,25.968571,42.528571,27.390000,41.030000,24.7,...,27,0,61200,19729,1,0,0,0,0,0
19729,2016-05-27 17:10:00,90,0,25.533333,46.860000,25.978000,42.534000,27.323333,41.090000,24.7,...,27,0,61800,19730,1,0,0,0,0,0
19730,2016-05-27 17:20:00,100,0,25.566667,46.560000,25.890000,42.025714,27.200000,41.163333,24.7,...,27,0,62400,19731,1,0,0,0,0,0
19731,2016-05-27 17:30:00,90,0,25.500000,46.500000,25.754000,42.080000,27.133333,41.223333,24.7,...,27,0,63000,19732,1,0,0,0,0,0
19732,2016-05-27 17:40:00,270,10,25.500000,46.596667,25.628571,42.768571,27.050000,41.690000,24.7,...,27,0,63600,19733,1,0,0,0,0,0
19733,2016-05-27 17:50:00,420,10,25.500000,46.990000,25.414000,43.036000,26.890000,41.290000,24.7,...,27,0,64200,19734,1,0,0,0,0,0
19734,2016-05-27 18:00:00,430,10,25.500000,46.600000,25.264286,42.971429,26.823333,41.156667,24.7,...,27,0,64800,19735,1,0,0,0,0,0


In [5]:
data1 = df.drop('Appliances', axis=1)


In [6]:
y= df["Appliances"]
y.index = df["id"]
y

id
1         60
2         60
3         50
4         50
5         60
6         50
7         60
8         60
9         60
10        70
11       230
12       580
13       430
14       250
15       100
16       100
17        90
18        70
19        80
20       140
21       120
22       190
23       110
24       110
25       110
26       110
27       100
28       100
29       100
30       100
        ... 
19706    280
19707    240
19708    250
19709    220
19710    230
19711    190
19712    160
19713     80
19714     60
19715     60
19716     60
19717     60
19718     70
19719     80
19720     80
19721     70
19722    100
19723    100
19724     90
19725    100
19726    220
19727    180
19728    120
19729    110
19730     90
19731    100
19732     90
19733    270
19734    420
19735    430
Name: Appliances, Length: 19735, dtype: int64

In [7]:
extracted_features = extract_features(data1, column_id="id", column_sort="date",show_warnings=False, default_fc_parameters=MinimalFCParameters())
extracted_features

Feature Extraction: 100%|██████████████████████████████████████████████████████████████| 20/20 [02:16<00:00,  6.43s/it]


variable,Friday__length,Friday__maximum,Friday__mean,Friday__median,Friday__minimum,Friday__standard_deviation,Friday__sum_values,Friday__variance,NSM__length,NSM__maximum,...,rv2__sum_values,rv2__variance,weekendstatus__length,weekendstatus__maximum,weekendstatus__mean,weekendstatus__median,weekendstatus__minimum,weekendstatus__standard_deviation,weekendstatus__sum_values,weekendstatus__variance
id,,,,,,,,,,,,,,,,,,,,,
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,61200.0,...,13.275433,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,61800.0,...,18.606195,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,62400.0,...,28.642668,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,63000.0,...,45.410389,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,63600.0,...,10.084097,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,64200.0,...,44.919484,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,64800.0,...,47.233763,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,65400.0,...,33.039890,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,66000.0,...,31.455702,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
impute(extracted_features)
features_filtered = select_features(extracted_features, y)

In [11]:
len(features_filtered)

19735

In [12]:
features_filtered

variable,T9__mean,T9__median,T9__minimum,T9__maximum,T9__sum_values,month__sum_values,month__minimum,month__median,month__mean,month__maximum,...,Wednesday__sum_values,Wednesday__minimum,Wednesday__maximum,Wednesday__mean,Wednesday__median,Thursday__mean,Thursday__maximum,Thursday__sum_values,Thursday__minimum,Thursday__median
id,,,,,,,,,,,,,,,,,,,,,
1,17.033333,17.033333,17.033333,17.033333,17.033333,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,17.066667,17.066667,17.066667,17.066667,17.066667,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,17.000000,17.000000,17.000000,17.000000,17.000000,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,17.000000,17.000000,17.000000,17.000000,17.000000,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,17.000000,17.000000,17.000000,17.000000,17.000000,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,17.000000,17.000000,17.000000,17.000000,17.000000,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,17.000000,17.000000,17.000000,17.000000,17.000000,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,17.000000,17.000000,17.000000,17.000000,17.000000,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,17.000000,17.000000,17.000000,17.000000,17.000000,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
y

id
1         60
2         60
3         50
4         50
5         60
6         50
7         60
8         60
9         60
10        70
11       230
12       580
13       430
14       250
15       100
16       100
17        90
18        70
19        80
20       140
21       120
22       190
23       110
24       110
25       110
26       110
27       100
28       100
29       100
30       100
        ... 
19706    280
19707    240
19708    250
19709    220
19710    230
19711    190
19712    160
19713     80
19714     60
19715     60
19716     60
19717     60
19718     70
19719     80
19720     80
19721     70
19722    100
19723    100
19724     90
19725    100
19726    220
19727    180
19728    120
19729    110
19730     90
19731    100
19732     90
19733    270
19734    420
19735    430
Name: Appliances, Length: 19735, dtype: int64

In [16]:
from sklearn.preprocessing import MinMaxScaler
a1 = y.values
a2 = a1.reshape(len(a1),1)
scaler = MinMaxScaler(feature_range=(0, 1))
df_1 = scaler.fit_transform(a2)


C:\Users\chethan\Anaconda3\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


In [17]:
df_1

array([[0.04672897],
       [0.04672897],
       [0.03738318],
       ...,
       [0.24299065],
       [0.38317757],
       [0.39252336]])

In [18]:
df_2 = scaler.fit_transform(features_filtered)

In [20]:
df_2.shape

(19735, 175)

In [21]:
x = df_2
y = df_1

In [22]:
Ytrainsize = int(len(y) * 0.7)
Ytestsize = (len(y) - Ytrainsize)
train_Y, test_Y = y[0:Ytrainsize,:], y[Ytrainsize:len(df_1),:]
print(len(train_Y) , len(test_Y))

Xtrainsize = int(len(x) * 0.7)


Xtestsize = (len(x) - Xtrainsize)
train_X, test_X = x[0:Xtrainsize,:], x[Xtrainsize:len(df_2),:]
print(len(train_X) , len(test_X))

13814 5921
13814 5921


In [23]:
train_Y=train_Y.ravel()


In [24]:
from sklearn.ensemble import RandomForestRegressor
regressor = RandomForestRegressor(n_estimators = 40, random_state = 2)
regressor.fit(train_X, train_Y)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=40, n_jobs=1,
           oob_score=False, random_state=2, verbose=0, warm_start=False)

In [27]:
#from sklearn.feature_selection import RFE
#selector = RFE(regressor, step=1)
#selector = selector.fit(train_X, train_Y)

In [29]:
y_pred = regressor.predict(test_X)
y_pred=y_pred.reshape(-1,1)
y_pred

array([[0.1478972 ],
       [0.15093458],
       [0.16308411],
       ...,
       [0.35514019],
       [0.34929907],
       [0.34929907]])

In [30]:
from math import sqrt
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from statistics import mean 
import statistics
from sklearn import preprocessing

In [31]:
def mae(org,pred):
    return mean_absolute_error(org,pred)
def rmse(org,pred):
    return np.sqrt(mean_squared_error(org,pred))
def r2s(org,pred):
    return r2_score(org,pred)
def mape(org,pred):
    y_true, y_pred = np.array(org), np.array(pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [32]:
def scores(model,yorg,ypred):
    print("The scores for the model : ",model)
    print("MAE    :    ",mae(yorg,ypred))
    print("RMSE   :    ",rmse(yorg,ypred))
    print("R2     :    ",r2s(yorg,ypred))
    print("MAPE   :    ",mape(yorg,ypred))

In [35]:
y_pred

array([[0.1478972 ],
       [0.15093458],
       [0.16308411],
       ...,
       [0.35514019],
       [0.34929907],
       [0.34929907]])

In [36]:
test_Y

array([[0.10280374],
       [0.13084112],
       [0.08411215],
       ...,
       [0.24299065],
       [0.38317757],
       [0.39252336]])

In [37]:
train_Y

array([0.04672897, 0.04672897, 0.03738318, ..., 0.17757009, 0.08411215,
       0.08411215])

In [38]:
y_pred_train = regressor.predict(train_X)

In [39]:
y_pred_train

array([0.05303738, 0.04976636, 0.03831776, ..., 0.18060748, 0.13341121,
       0.12102804])

In [40]:
print("Scores for train")
scores("RFE - tsfresh",train_Y,y_pred_train)

Scores for train
The scores for the model :  RFE - tsfresh
MAE    :     0.01096779442229135
RMSE   :     0.02358161928927214
R2     :     0.9442380547526009


C:\Users\chethan\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: RuntimeWarning: divide by zero encountered in true_divide
  if __name__ == '__main__':


MAPE   :     inf


In [41]:
y_pred

array([[0.1478972 ],
       [0.15093458],
       [0.16308411],
       ...,
       [0.35514019],
       [0.34929907],
       [0.34929907]])

In [42]:
test_Y

array([[0.10280374],
       [0.13084112],
       [0.08411215],
       ...,
       [0.24299065],
       [0.38317757],
       [0.39252336]])

In [44]:
print("Scores for train")
scores("RFE - tsfresh",test_Y.ravel(),y_pred.ravel())

Scores for train
The scores for the model :  RFE - tsfresh
MAE    :     0.12568842564166507
RMSE   :     0.16319956141911257
R2     :     -2.63619960158934
MAPE   :     225.12929503336022
